In [16]:
from pyspark import SparkConf , SparkContext

In [17]:
sc.stop()
conf = SparkConf().setMaster("local").setAppName("wordcount")
sc = SparkContext(conf = conf)
sc

<SparkContext master=local appName=wordcount>

input 點跟pagerank作為(key,value)，用tab分開

In [18]:
def mapper1try(line):
        wordlist = line.split(" ")
        newitem = []
        maplist = []

        for item in wordlist:
                c = item.split("\t")
                for unit in c:
                    unit = int(unit)
                    newitem.append(unit)

                maplist.append((newitem[0],newitem[1]))
        return maplist

mapreduce,再將同個點會連到的點map成lines2，原本的key成為lines3

In [19]:
lines1 = sc.textFile("/Users/huangyushu/Desktop/in.txt").flatMap(mapper1try)
lines2 = lines1.groupByKey().map(lambda x : (list(x[1])))
lines3 = lines1.map(lambda x : (x[0]))
lines3 = lines3.distinct()
print(lines2.collect())
print(lines3.collect())

[[2, 3], [4], [1, 4, 5], [1, 4]]
[1, 2, 3, 5]


這裡用到達的點中的最大值作為點的數量的上標，因為可以有deadend所以要用可到達的最大點index作為max

In [20]:
try1 = lines1.reduceByKey(max)
print(try1.collect())
try1 = try1.max(lambda x:x[1])
try1[1]

[(1, 3), (2, 4), (3, 5), (5, 4)]


5

這裡用到達的點中的最大值作為點的數量的下標，因為可以有deadend所以要用可到達的最小點index作為min

In [21]:
try2 = lines1.reduceByKey(min)
print(try2.collect())
try2 = try2.min(lambda x:x[1])
# try1 = try1.values()
try2[1]
# type(try1)

[(1, 2), (2, 4), (3, 1), (5, 1)]


1

pagerank matrix 中：
總數
beta值
後面常數項

In [22]:
n = try1[1]-try2[1]+1
beta = 0.8
dump = (1-beta)/n

mapper:將每筆資料裡面的每個子項依序標數字n次
mapkey:將每筆資料標數字n次
mapone:將資料變成key為1之value

In [23]:
def mapper(line):
    map = []
    mapkey = []
    for i in range(n):
        map.append((i,line[i]))  
    return map

In [24]:
def mapkey(line):
    map = []
    mapkey = []
    for i in range(n):
        map.append((i,line))  
    return map

In [25]:
def mapone(line):
    map = []
    map.append((1,line))  
    return map

用兩個rdd做出n x n的rdd，adj與key對應到在matrix的哪個位置 i,j = (點,到達的點) 應該設什麼val，也就是這個點的父點之1/out degree

In [26]:
lines = lines2.zip(lines3)
lines.collect()

[([2, 3], 1), ([4], 2), ([1, 4, 5], 3), ([1, 4], 5)]

In [27]:
def create(adj,key):
    adj = adj.collect()
    key = key.collect()
    matrix = [[0 for _ in range(n)] for _ in range(n)]
    for col, line in enumerate(adj):
        for index in line:
            matrix[index-try2[1]][key[col]-try2[1]] = 1/(len(line))
    return matrix

將要相乘的兩個rdd傳入，分別攤平後以以key排序，變成nxn比相對應的資料再zip起來，做相乘計算後跟排序好的key再合併，直接做mapreduce將值sum求出
完成兩個rdd矩陣相乘

In [28]:
def multiply(na,l):
    l = l.flatMap(mapkey)
    l = l.sortByKey()
    l = l.values()
    
    na2 = na.flatMap(mapper)
    na3 = na2.sortByKey()
    na3 = na3.keys()
    na2 = na2.values()
    na2 = na2.zip(l)
    
    na2 = na2.map(lambda x:(x[0]*x[1]))
    na2 = na3.zip(na2)
    
    na2 = na2.reduceByKey(lambda x,y:(x+y))
    na2 = na2.values()
    return na2

In [29]:
ma = create(lines2,lines3)

In [30]:
ma = sc.parallelize(ma)

這裡算出A矩陣na ＝bm+(1-b)*1/n，相乘的方法同上面的map

In [31]:
ma = ma.flatMap(mapper)

In [32]:
sol = ma.sortByKey()
sol = sol.keys()

In [33]:
na = ma.map(lambda x:(x[1]*beta+dump))

In [34]:
na = sol.zip(na)

In [35]:
na = na.groupByKey().map(lambda x : (list(x[1])))

初始pagerank value：1/n

In [36]:
v2 = [(1/n) for _ in range(n)] 
v2 = sc.parallelize(v2)

20iterations，有deadend:r = r + (1-s/n)

In [37]:
    for i in range(20):
        
        a = []
        v1 = v2
        v2 = multiply(na,v1)
        
        s = v2.sum()
        v0 = v2.flatMap(mapone)
        v0 = v0.map(lambda x:(x[1]+(1-s)/n))
        v2 = v0
        print(v2.collect())


[0.20533333333333334, 0.152, 0.152, 0.3653333333333334, 0.12533333333333332]
[0.18911999999999998, 0.18058666666666667, 0.18058666666666667, 0.31072, 0.13898666666666665]
[0.19346631111111112, 0.1653632, 0.1653632, 0.33793564444444446, 0.13787164444444444]
[0.19331521422222223, 0.17145622755555556, 0.17145622755555556, 0.32560577422222226, 0.13816655644444445]
[0.1930852071348148, 0.16942300956444442, 0.16942300956444442, 0.33025018917925925, 0.137818584557037]
[0.1931469333086815, 0.17007411312260745, 0.17007411312260745, 0.328685340960237, 0.13801949948586667]
[0.1931505511806799, 0.16984842787711052, 0.16984842787711052, 0.32920984167876594, 0.13794275138633325]
[0.19314358932369863, 0.16993379514087448, 0.16993379514087448, 0.329022331625387, 0.13796648876916534]
[0.19314584727196127, 0.1699010087895414, 0.1699010087895414, 0.32909288338466086, 0.13795925176429513]
[0.1931454977244748, 0.16991320025033024, 0.16991320025033024, 0.32906630475610793, 0.13796179701875677]
[0.1931455143

output:用c算出在小數點後幾位開始有數字，四捨五入round到開始有數字的第三位

In [38]:
    b = []
    for item in v2.collect():
        c=0
        fig = item
        while(fig<1):
            fig = fig*10
            c=c+1
        item = round(item,c+2)
        b.append(item)
        
    lines = b

output print，前面加上vertex

In [39]:
ans = []
for key, val in enumerate(lines):
    ans.append((key+try2[1],val))

排序並寫入outputfile

In [40]:
ans = sc.parallelize(ans).sortBy(lambda x: x[1], ascending=False)
ans.collect()

[(4, 0.329), (1, 0.193), (2, 0.17), (3, 0.17), (5, 0.138)]

In [41]:
path = '/Users/huangyushu/Desktop/out.txt'
data=open(path,'w+') 
for i in range(n):
        print('%d' % ans.collect()[i][0],'\t','%f' % ans.collect()[i][1],file=data)
data.close()